In [16]:
# We've gotten around parsing and cropping the image. Now we want to set
# up for the mock image reconstruction task by fragmenting the image
# into a mosaic which we'll shuffle.

# Importing tools
from PIL import Image, ImageFilter
import numpy as np
from numpy import random
import math

#Read and display image in-notebook
im = Image.open( 'TestImages/kitty.jpg' ).rotate(-90)
#im

In [2]:
# converting image data to numpy array
pix = np.array(im);

In [3]:
# analyzing dimensions of "arrayed" image ()
pix.shape

(3024, 4032, 3)

In [4]:
%%time
# timing subset execution with cell magic

# We've noticed that the rotated image has a padded black border,
# which we want to remove for reconstruction (artificial original
# border = ability to 'cheat' the reconstruction algo!)
RL = 0
RU = 3024
CL = 550
CU = 3500
pixCropped = pix[RL:RU,CL:CU,0:3]

Wall time: 501 µs


In [5]:
# printing cropped image
imCropped = Image.fromarray(pixCropped,'RGB')
#imCropped

In [6]:
# analyzing size of cropped image
pixCropped.shape

(3024, 2950, 3)

In [99]:
# Now we want to let the user/test administrator pick a number of
# row partitions and column partitions for fragging the image.

# test administrator-defined partitions
rowPartitions = 5
colPartitions = 3

# storing values for # total rows, cols (pixels)
pixelRows = pixCropped.shape[0];
pixelCols = pixCropped.shape[1];

# approximate row, column length of partitions
rowInterval = math.ceil(pixelRows/rowPartitions);
colInterval = math.ceil(pixelCols/colPartitions);

# preparing and printing a preliminary impression of partitioning
pixPartitioned1 = np.copy(pixCropped)

# row partitions
for i in range(1,rowPartitions):
    rowBegin = i*rowInterval-4
    rowEnd   = i*rowInterval+5
    pixPartitioned1[rowBegin:rowEnd,:,:] = 255
        
# column partitions
for j in range(1,colPartitions):
    colBegin = j*colInterval-4
    colEnd   = j*colInterval+5
    pixPartitioned1[:,colBegin:colEnd,:] = 255

# printing out a concept illustration of the fragmentation
imPartitioned1 = Image.fromarray(pixPartitioned1)
#imPartitioned1

In [100]:
# Now, for purposes of a 'fair' starting point for the reconstruction
# algo, we'll want to make sure the size of each of these fragments is
# either uniform or randomized in such a way that their size doesn't give
# the algo any clue as to where they came from.

# For instance, if our rows and cols aren't evenly divisible by the #
# of partitions specified, the algo will have a clue as to which frags
# correspond to a bottom or right edge or a lower-right corner.

# Also, for purposes of setting up a more challenging problem, we'll set
# an amount of overlap (with surrounding fragments) for every frag, and
# also an offset that is NOT uniform among the fragments. We'll displace
# the fragment by this amount.

# Important: so this doesn't get TOO crazy, we'll make sure the possible
# offsets fit within the overlap buffer (e.g. if we are allowed to
# displace a fragment by 5 pixels in either the x- or y-direction, we'd
# better make sure our overlap buffer is 11 or more, or else adjacent
# frags may not have any overlap (which we'll rely on for our simple
# reconstruction algorithm))

In [101]:
# determining number of total image fragments
numFragments = rowPartitions*colPartitions

# Suppose our fragments in the 3,2 partition example are indexed like:
# 0 1
# 2 3
# 4 5

# or, flattened:
# 0 1 2 3 4 5

# we can shuffle these flattened indices with a random permutation:
shufFragIndices = np.random.permutation(numFragments)
print(shufFragIndices)

[ 7  4  9  5  6  0 14 11  2  8 13 10 12  3  1]


In [102]:
# now we'll define our overlap and offsets.
offsetMax = 2

overlapPix = 10

# ensuring overlap algo will work (sufficient overlap for offsets)
overlapPix = max(overlapPix,3*offsetMax)

# using these parameters, we'll define an array of x,y pixel
# offsets for each image fragment
xyOffsets = np.random.randint(offsetMax+1,size=(numFragments,2))

In [112]:
# now we'll define our overlap and offsets.
offsetMax = 2

overlapPix = 10

# ensuring overlap algo will work (sufficient overlap for offsets)
overlapPix = max(overlapPix,3*offsetMax)

# using these parameters, we'll define an array of x,y pixel
# offsets for each image fragment
xyOffsets = np.random.randint(2*offsetMax+1,size=(numFragments,2))-offsetMax

In [104]:
# We'll initialize and fill a 4-D (!) array to handle all the frags in the shuffled
# order given by shufFragIndices.
pixFrags = np.zeros((numFragments,rowInterval + 2*overlapPix,colInterval + 2*overlapPix,3),dtype='uint8')

for i in range(0,numFragments):
    # block indices for starting fragmentation
    blockRowInd = shufFragIndices[i] // colPartitions
    blockColInd = shufFragIndices[i] % colPartitions
    
    # defining part of original image given over to fragment 'i'
    # (including overlap and x/y offset)
    startRow = blockRowInd*rowInterval - overlapPix + xyOffsets[i,0]
    endRow = startRow + rowInterval + 2*overlapPix
    startCol = blockColInd*colInterval - overlapPix + xyOffsets[i,1]
    endCol = startCol + colInterval + 2*overlapPix
    
    # culling interval back in the case of original image overflow/underflow
    if startRow < 0:
        startRow = abs(xyOffsets[i,0])
        endRow = rowInterval + 2*overlapPix + abs(xyOffsets[i,0])
    elif endRow > pixCropped.shape[0]:
        startRow = pixCropped.shape[0] - rowInterval - 2*overlapPix - abs(xyOffsets[i,0])
        endRow = pixCropped.shape[0] - abs(xyOffsets[i,0])
        
    if startCol < 0:
        startCol = abs(xyOffsets[i,1])
        endCol = colInterval + 2*overlapPix + abs(xyOffsets[i,1])
    elif endCol > pixCropped.shape[1]:
        startCol = pixCropped.shape[1] - colInterval - 2*overlapPix - abs(xyOffsets[i,1])
        endCol = pixCropped.shape[1] - abs(xyOffsets[i,1])
    
    # now we fill in pixFrags (preallocated shape) with shuffled and offset data from
    # pixCropped.
    pixFrags[i,:,:,:] = pixCropped[startRow:endRow,startCol:endCol,:]

In [105]:
# We can now assemble a shuffled view of the original photo!
pixShuffled = np.zeros((rowInterval*rowPartitions,colInterval*colPartitions,3),dtype = 'uint8')

for i in range(0,numFragments):
    blockRowInd = i // colPartitions
    blockColInd = i % colPartitions
    
    startRow = rowInterval*blockRowInd
    endRow = startRow + rowInterval
    
    startCol = colInterval*blockColInd
    endCol = startCol + colInterval
     
    fragRS = overlapPix
    fragRE = rowInterval + overlapPix
    
    fragCS = overlapPix
    fragCE = colInterval + overlapPix
    
    pixShuffled[startRow:endRow,startCol:endCol,:] = pixFrags[i,fragRS:fragRE,fragCS:fragCE,:]

In [108]:
# printing out a concept illustration of the shuffled image
imShuffled1 = Image.fromarray(pixShuffled)
#imShuffled1

In [107]:
# shuffled image looks good! time to move on to an elementary reconstruction algo.